In [3]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis_lvm import run_ibis_lvm

from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [4]:
log_dir = './log/20210101_210037_ez_lvm/'

## Create Data

In [5]:
exp_data = load_obj('data', log_dir)

## Run HMC

In [33]:
ps = load_obj( 'ps_hmc', log_dir)

In [34]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]).reshape(1000,6,1)) 
df['source'] = 'hmc'
plot_density(df) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.Chart(...)

## Run IBIS

In [35]:
# load existing results

particles = load_obj('particles', log_dir)
t = load_obj('t', log_dir)
print(t)

170


## Plot Both results

In [30]:
# have to resample particles to get rid of weights
# particles = ibis['particles']
particles.resample_particles()

In [31]:
np.mean(np.squeeze(particles.particles['beta']),0)

array([-0.01997524, -0.03067603, -0.08263782, -0.0105808 , -0.04299418,
       -0.11092549])

In [32]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]).reshape(1000,6,1)) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.Chart(...)

In [28]:
param = 'alpha'
df = get_post_df(np.squeeze(ps[param])) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 

alt.Chart(...)